In [0]:
!pip install transformers

     |████████████████████████████████| 317kB 3.5MB/s 
     |████████████████████████████████| 860kB 41.0MB/s 
     |████████████████████████████████| 1.0MB 34.8MB/s 
     |████████████████████████████████| 645kB 37.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=9cdb0710c0f3a7005eed1c380bca7dc3e78f0a66679881e36d15ad6f27c54c90
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
import pickle
import torch
import numpy as np
import torch.optim as optim
# from transformers import BertConfig, BertModel, BertForMaskedLM
from transformers import GPT2Config, GPT2Model, GPT2LMHeadModel

from IPython.display import clear_output

# import matplotlib.pyplot as plt
# from transformers import *

In [0]:
d = "drive/My Drive/Colab Notebooks/smaug/data"
p = os.path.join(d, "CDS_singlestrain_threeprime.pkl")
print(os.listdir(d))

['CDS_singlestrain_threeprime.pkl', 'models', 'config.json', 'pytorch_model.bin']


In [0]:
with open(p, 'rb') as f:
  CDS = pickle.load(f)

In [0]:
print(type(CDS))
print(len(CDS))
print(CDS[100])

<class 'list'>
9304394
HVSLLSVHAAVAVWRKKRQMYIDQYCVRGTKLTNAEKFVFTMYATVVGIKEDLMRVDASDINVSLIEQRRLNRIVDRRTMKNGDPSILIFDNAFIRELTVSDKSPYSRIWDENMRKLTEVSKNQAHLQCTFVMVLSFLLIAKM


In [0]:
# lengths = [len(s) for s in CDS]
# meanlen = np.mean(lengths)
# medlen = np.median(lengths)
# minlen = np.min(lengths)
# maxlen = np.max(lengths)
# print(meanlen, medlen, minlen, maxlen)

In [0]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [0]:
# # custom parameters for BERT model
# vocab_size = 12 # Vocabulary size of inputs_ids in BertModel. default=30522
# hidden_size = 768 # Size of the encoder layers and the pooler layer, default=768
# num_hidden_layers = 12 # Number of hidden layers in the Transformer encoder. default=12
# num_attention_heads = 12 # Number of attention heads for each attention layer in the Transformer encoder, default=12
# intermediate_size = 3072 # The size of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder. default=3072
# hidden_act = "gelu" # The non-linear activation function (function or string) in the encoder and pooler. If string, “gelu”, “relu”, “swish” and “gelu_new” are supported. default="gelu"
# hidden_dropout_prob = 0.1 # The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler. default=0.1
# attention_probs_dropout_prob = 0.1 # The dropout ratio for the attention probabilities. default=0.1
# max_position_embeddings = 512 # The maximum sequence length that this model might ever be used with. Typically set this to something large just in case (e.g., 512 or 1024 or 2048). default=512
# type_vocab_size = 2 # 1 # The vocabulary size of the token_type_ids passed into BertModel. default=2
# initializer_range = 0.02 # The sttdev of the truncated_normal_initializer for initializing all weight matrices. default=0.02
# layer_norm_eps = 1e-12 # The epsilon used by LayerNorm. default=1e-12


# config = BertConfig(vocab_size_or_config_json_file=vocab_size,
#                     hidden_size=hidden_size,
#                     num_hidden_layers=num_hidden_layers,
#                     num_attention_heads=num_attention_heads,
#                     intermediate_size=intermediate_size,
#                     hidden_act=hidden_act,
#                     hidden_dropout_prob=hidden_dropout_prob,
#                     attention_probs_dropout_prob=attention_probs_dropout_prob,
#                     max_position_embeddings=max_position_embeddings,
#                     type_vocab_size=type_vocab_size,
#                     initializer_range=initializer_range,
#                     layer_norm_eps=layer_norm_eps)

# model = BertForMaskedLM(config)

# print(model)
# model.to('cuda')

In [0]:
# custom parameters for GPT2 model
vocab_size = 22
max_position_embeddings = 40 # 1024
n_ctw = max_position_embeddings # 1024
n_embd = 36 # 768
n_layer = 12 # 12
n_head = 12 # 12


config = GPT2Config(vocab_size_or_config_json_file=vocab_size,
                    n_positions=max_position_embeddings,
                    n_ctw=n_ctw,
                    n_embd=n_embd,
                    n_layer=n_layer,
                    n_head=n_head)

model = GPT2LMHeadModel(config)

print(model)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(22, 36)
    (wpe): Embedding(40, 36)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((36,

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(22, 36)
    (wpe): Embedding(40, 36)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((36,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((36,

In [0]:
# encode data as GPU tensors
max_aa_seq_length = max_position_embeddings


def tokenize_aa_seq_murphy10(aa_seq):
    table = {"L":1,
             "V":1,
             "I":1,
             "M":1,
             "C":2,
             "A":3,
             "G":4,
             "S":5,
             "T":5,
             "P":6,
             "F":7,
             "Y":7,
             "W":7,
             "E":8,
             "D":8,
             "N":8,
             "Q":8,
             "K":9,
             "R":9,
             "H":10,
             "X":0, # get rid of those
             "B":0,
             "*":0}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized

def tokenize_aa_seq(aa_seq):
    table = {"L":1,
             "V":2,
             "I":3,
             "M":4,
             "C":5,
             "A":6,
             "G":7,
             "S":8,
             "T":9,
             "P":10,
             "F":11,
             "Y":12,
             "W":13,
             "E":14,
             "D":15,
             "N":16,
             "Q":17,
             "K":18,
             "R":19,
             "H":20,
             "X":0, # get rid of those
             "B":0,
             "*":0}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized

# tokens = [tokenize_aa_seq_murphy10(seq) for seq in CDS[:1000]] ######################################
tokens = [tokenize_aa_seq(seq) for seq in CDS[:100000]] ######################################

tokens_tensor = torch.zeros(len(tokens), max_aa_seq_length, dtype=torch.long)
for i in range(len(tokens)):
    l = len(tokens[i]) # scuff way to ensure fit in tensor, TODO build correctly sized data set and split into train test
    if l > max_aa_seq_length:
        l = max_aa_seq_length
    for j in range(l):
        tokens_tensor[i][j] += tokens[i][j]

tokens_tensor = tokens_tensor.to('cuda')

print(tokens_tensor)
print(tokens_tensor.shape)

tensor([[18,  1,  4,  ...,  6, 11,  8],
        [ 6, 17,  6,  ..., 11,  5,  7],
        [15, 19,  1,  ..., 10, 14, 20],
        ...,
        [15, 18,  6,  ..., 11, 15, 18],
        [18, 14, 14,  ...,  3, 14,  9],
        [ 6, 16,  6,  ...,  3,  8,  5]], device='cuda:0')
torch.Size([100000, 40])


In [0]:
dtrain = tokens_tensor[:int(0.8*len(tokens_tensor))] #TODO do an actual random selection on better data
dvalid = tokens_tensor[int(0.8*len(tokens_tensor)):]
print(len(dtrain))
print(len(dvalid))

80000
20000


In [0]:
# optimizer = optim.SGD(model.parameters(), lr=0.0001)#, momentum=args.momentum)
optimizer = optim.AdamW(model.parameters())
# optimizer = optim.Adam(model.parameters())

In [0]:
def savemodel():
    # save model
    # Load the Drive helper and mount
    from google.colab import drive

    # This will prompt for authorization.
    drive.mount('/content/drive')

    modeldir = "drive/My Drive/Colab Notebooks/smaug/data/models"
    model.save_pretrained(modeldir)
    print(os.listdir(modeldir))
    # p = os.path.join(d, "CDS_singlestrain_threeprime.pkl")

In [30]:
# train model with single aa masked at a time ##### batch
model.train()

batch_size = 2**10

optimizer.zero_grad()
np.random.seed(42424)
for i in range(100000):
    if i%500==10:
        savemodel()
    optimizer.zero_grad()

    select_idx = np.random.randint(0, len(dtrain), batch_size)

#     input_ids = dtrain[select_idx].unsqueeze(0) # singleton
    input_ids = dtrain[select_idx]
    
    
    outputs = model(input_ids, labels=input_ids)
    loss, prediction_scores = outputs[:2]
    
    loss.backward()
    optimizer.step()
    
    clear_output(wait=True)
    for k in range(0, 10):
#         print(i, torch.argmax(prediction_scores[0,k-1]).item(), input_ids[0,k].item(), "\t", loss.item()) #TODO figure out why GPT2 only offsets sometimes
        print(i, "\t", torch.argmax(prediction_scores[0,k-1]).item(), "\t", input_ids[0,k].item(), "\t", loss.item())
#         print(prediction_scores[0])
    


RuntimeError: ignored

In [0]:
# save model
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

modeldir = "drive/My Drive/Colab Notebooks/smaug/data/models"
model.save_pretrained(modeldir)
print(os.listdir(modeldir))
# p = os.path.join(d, "CDS_singlestrain_threeprime.pkl")

Mounted at /content/drive
['10-24-19_small', 'config.json', 'pytorch_model.bin']


In [0]:
# # TRIVIAL TASK: learn a single gene, should do perfectly
# # train model with single aa masked at a time ##### batch
# model.train()

# batch_size = 2**5

# # loss_mask = (torch.zeros(batch_size, max_position_embeddings)-1).long().to('cuda')
# optimizer.zero_grad()
# np.random.seed(2019)
# for i in range(10000):
# #     select_idx = np.random.randint(0, len(dtrain), batch_size)
# #     select_idx = np.random.randint(0, 1, batch_size)
#     select_idx = torch.zeros(batch_size).long() + 100
# #     print(select_idx)
# #     sys.exit()

# #     print(batch_size)
# #     sys.exit()
# #     input_ids = dtrain[select_idx].unsqueeze(0)
#     input_ids = dtrain[select_idx]
    
    
#     outputs = model(input_ids, labels=input_ids)
#     loss, prediction_scores = outputs[:2]
    
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
    
#     clear_output(wait=True)
#     for k in range(1, 10):
#         print(i, torch.argmax(prediction_scores[0,k-1]).item(), input_ids[0,k].item(), "\t", loss.item())
# #     print()
    


In [28]:
# evaluate the model using softmax proportion of correct label
model.eval()

np.random.seed(4242424)

maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(50):
    select_idx = np.random.randint(0,len(dvalid),1)[0]    
    input_ids = dvalid[select_idx].unsqueeze(0)
    # print(input_ids)
    
    for j in range(0, max_position_embeddings-1):
        tempval = 0
        tempval += input_ids[0, j+1].item()
#         print(tempval)
        if tempval != 0:
            logits = model(input_ids)[0][0,j]
            soft = torch.exp(logits)
            softproportion = (soft[tempval]/torch.sum(soft)).item() #######bug tempval not j
            softsum += softproportion
#             print(torch.sum(soft))
            maxsum += 1

            pred = torch.argmax(logits).item()
            actual = tempval
            n_correct += pred==actual
            n_total +=1
#             if j ==5:
#                 sys.exit()
            

correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

0.1194689525090731
0.20205128205128206


In [23]:
# evaluate the model using softmax proportion of correct label, ON NOISE
model.eval()

np.random.seed(4242424)

maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(100):
#     select_idx = np.random.randint(0,len(dvalid),1)[0]    
#     input_ids = dvalid[select_idx].unsqueeze(0)
    noise = np.random.randint(1, 20, max_position_embeddings).reshape(1,-1)
    input_ids = torch.from_numpy(noise).to('cuda')
#     print(input_ids)
    
    for j in range(max_position_embeddings):
        tempval = 0
        tempval += input_ids[0, j].item()
        if tempval != 0:
            logits = model(input_ids)[0][0,j]
            soft = torch.exp(logits)
            softproportion = (soft[tempval]/torch.sum(soft)).item()
            softsum += softproportion
            maxsum += 1

            pred = torch.argmax(logits).item()
            actual = tempval
            n_correct += pred==actual
            n_total +=1
            

correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

0.05510502388636814
0.0565


In [24]:
# evaluate using model that just predicts most common amino acid
model.eval()

np.random.seed(4242424)


maxsum = 0
softsum = 0
n_correct = 0
n_total = 0
for i in range(500):
    select_idx = np.random.randint(0,len(dvalid),1)[0]    
    input_ids = dvalid[select_idx].unsqueeze(0)
    
    for j in range(max_position_embeddings):
        tempval = 0
        tempval += input_ids[0, j].item()
        if tempval != 0:
            soft = torch.zeros(vocab_size)
            soft[1] += 1 ################
            softproportion = (soft[tempval]/torch.sum(soft)).item()
            softsum += softproportion
            maxsum += 1

            pred = torch.argmax(soft).item()

            actual = tempval
            n_correct += pred==actual
            n_total += 1
            
correctness = softsum/maxsum
print(correctness)

correctness_p = n_correct/n_total
print(correctness_p)

0.09955
0.09955


In [0]:
# evaluate model GPT2
model.eval()

np.random.seed(424242)

n_correct = 0
n_wrong = 0
for i in range(200):
    select_idx = np.random.randint(0,len(dvalid), 2)[0]    
    input_ids = dvalid[select_idx].unsqueeze(0)
    logits = model(input_ids, labels=input_ids)[1]
    prediction = torch.argmax(logits, 2)
#     print(logits.shape)
#     print(prediction)
#     print(input_ids)
    correct = torch.sum(prediction[0, 1:]==input_ids[0, 1:]).item()
    n_correct += correct # exclude first value because model is directionally causal
    n_wrong += (max_position_embeddings-1)-correct # max possible is max_position_embeddings-1
#     print(n_correct, n_wrong)

print(n_correct/(n_correct+n_wrong))

0.12


In [0]:
# evaluate model GPT2 ON NOISE, should be 0.05
model.eval()

np.random.seed(42)

n_correct = 0
n_wrong = 0
for i in range(1000):
    noise = np.random.randint(1, 20, max_position_embeddings).reshape(1,-1)
    input_ids = torch.from_numpy(noise).to('cuda')
    
    logits = model(input_ids, labels=input_ids)[1]
    prediction = torch.argmax(logits, 2)
#     print(prediction)
#     print(input_ids)
    correct = torch.sum(prediction[0, 1:]==input_ids[0, 1:]).item()
    n_correct += correct # exclude first value because model is directionally causal
    n_wrong += (max_position_embeddings-1)-correct # max possible score is max_position_embeddings-1
#     print(n_correct, n_wrong)

print(n_correct/(n_correct+n_wrong))

0.05333333333333334


In [0]:
# # evaluate model
# model.eval()

# np.random.seed(424242)
# predseq = []
# actualseq = []
# for i in range(20):
#     select_idx = np.random.randint(0,len(dvalid),1)[0]    
#     input_ids = dvalid[select_idx].unsqueeze(0)

    
# #     for j in range(max_position_embeddings):
#     for j in range(10):

#         tempval = 0
#         tempval += input_ids[0, j].item()
#         if tempval != 0:
#             input_ids[0, j] = vocab_size-1 # mask label is the highest vocab number, never present in data
#             predicted_aa = torch.argmax(model(input_ids)[0][0,j]).item() # TODO mask padding 0's for attention
# #             print(model(input_ids)[0][0,j])
# #             print(input_ids)
# #             sys.exit()
#             input_ids[0,j] = tempval # set masked value back to original value
# #             print(model(input_ids)[0][0,j])
#             predseq.append(predicted_aa)
#             actualseq.append(tempval)
#             print(i, predicted_aa,tempval)

# n_correct = 0
# n_wrong = 0
# for i, aa in enumerate(predseq):
#     if aa == actualseq[i]:
#         n_correct += 1
#     else:
#         n_wrong += 1
# print(n_correct/(n_correct+n_wrong))

0 11 10
0 9 20
0 4 9
0 4 7
0 1 1
0 12 7
0 4 7
0 8 1
0 6 1
0 12 1
1 11 10
1 9 18
1 4 1
1 11 19
1 1 12
1 12 7
1 4 9
1 8 19
1 6 11
1 12 3
2 11 11
2 8 6
2 4 14
2 11 13
2 1 14
2 12 10
2 4 9
2 8 6
2 6 8
2 12 19
3 11 14
3 8 10
3 4 2
3 11 1
3 1 13
3 12 8
3 4 9
3 8 19
3 6 11
3 12 19
4 11 19
4 8 5
4 4 15
4 4 17
4 1 5
4 12 16
4 4 7
4 8 19
4 6 6
4 12 4
5 11 8
5 8 2
5 4 9
5 11 3
5 1 2
5 12 7
5 4 5
5 8 17
5 6 6
5 12 1
6 11 20
6 8 8
6 4 7
6 11 4
6 1 1
6 12 18
6 4 14
6 8 2
6 6 6
6 12 3
7 11 18
7 8 6
7 4 9
7 4 10
7 1 4
7 12 2
7 4 11
7 8 14
7 6 6
7 12 7
8 11 6
8 9 15
8 4 10
8 4 1
8 1 6
8 12 6
8 4 3
8 8 2
8 6 4
8 12 3
9 11 10
9 9 1
9 4 10
9 11 9
9 1 10
9 12 17
9 4 16
9 8 8
9 6 6
9 12 10
10 11 19
10 8 17
10 4 3
10 4 14
10 1 9
10 12 2
10 4 2
10 8 14
10 6 9
10 12 18
11 11 19
11 8 6
11 4 15
11 4 3
11 1 6
11 12 6
11 4 17
11 8 6
11 6 6
11 12 14
12 11 6
12 9 8
12 4 16
12 4 1
12 1 19
12 12 13
12 4 3
12 8 8
12 6 6
12 12 16
13 11 1
13 8 18
13 4 17
13 11 14
13 1 1
13 12 14
13 4 10
13 8 1
13 6 2
13 12 14
14 11 15
14

In [0]:
    
    
    

#     for j in range(max_position_embeddings):
#         tempval = torch.zeros(batch_size).long()

#         for k in range(batch_size):
#             tempval[k] = input_ids[k, j].item()
        
#         optimizer.zero_grad()

# #         loss_mask[:, j] = tempval # calculate loss based only on masked amino acid
      
# #         input_ids[:, j] = vocab_size-1 # mask label is the highest vocab number, never present in data

#         outputs = model(input_ids, labels=) #TODO mask padding 0's for attention
# #         print(outputs)

#         loss, prediction_scores = outputs[:2]
#         print(outputs.shape)
#         sys.exit()
# #         loss, prediction_scores = outputs

        
#         for k in range(batch_size):
#             print(i, torch.argmax(prediction_scores[k,j]).item(), loss_mask[k,j].item(), "\t", loss.item())
#         print()
        
#         loss.backward()
#         optimizer.step()

#         for k in range(batch_size):
#             input_ids[k, j] = tempval[k] # set masked value back to original value
#             loss_mask[k, j] = -1

In [0]:
# # train model with single aa masked at a time
# model.train()
# loss_mask = (torch.zeros(max_position_embeddings)-1).long().to('cuda')

# np.random.seed(2019)
# for i in range(1000):
#     select_idx = np.random.randint(0,len(dtrain),1)[0]    
#     input_ids = dtrain[select_idx].unsqueeze(0)

#     for j in range(max_position_embeddings):
#         tempval = 0
#         tempval += input_ids[0, j].item()
#         if tempval != 0:
#             optimizer.zero_grad()

#             loss_mask[j] = tempval # calculate loss based only on masked amino acid
#             input_ids[0, j] = vocab_size-1 # mask label is the highest vocab number, never present in data

#             outputs = model(input_ids, masked_lm_labels=loss_mask) #TODO mask padding 0's for attention
#             loss, prediction_scores = outputs[:2]
#             print(i, torch.argmax(prediction_scores[0,j]).item(), loss.item())
#             loss.backward()
#             optimizer.step()

#             input_ids[0,j] = tempval # set masked value back to original value
#             loss_mask[j] = -1

# #             if j==10:
# #                 break
#         else:
#             break # stop training at end of protein sequence

# #     break

torch.Size([1, 100, 22])
1


In [0]:
def evaluate(model, data):
    model.eval()
    acc_list = []
    for d in data:
        outputs = model(d.unsqueeze(0), masked_lm_labels=d.unsqueeze(0))
        loss, prediction_scores = outputs[:2]
        
        predicted_index = torch.argmax(prediction_scores, dim=2)
        n_correct = torch.sum(predicted_index==d).item()
#         n_possible = torch.sum(d!=0).item()
        n_possible = len(predicted_index[0])
        acc = n_correct/n_possible
        acc_list.append(acc)
        
    return(np.mean(acc_list))

In [0]:
# train model
batch_size = 1

optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
# optimizer = optim.AdamW(model.parameters())
_max_select = len(dtrain)


# single step to non-zero weights
optimizer.zero_grad()
select_idx = np.random.randint(0, _max_select, batch_size)
outputs = model(dtrain[select_idx], masked_lm_labels=dtrain[select_idx])
loss, prediction_scores = outputs[:2]
loss.backward()
optimizer.step()

# # evaluate before training
# acc_train = evaluate(model, dtrain)
# acc_valid = evaluate(model, dvalid)
# print(acc_train)
# print(acc_valid)


# train
model.train()
np.random.seed(2019)
for i in range(100):
    optimizer.zero_grad()
    select_idx = np.random.randint(0, _max_select, batch_size)
    input_ids = tokens_tensor[select_idx]
    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    print(i, loss.item())
    loss.backward()
    optimizer.step()
    
#     loss.backward(retain_graph=True)



In [0]:
# after training
acc_train = evaluate(model, dtrain)
acc_valid = evaluate(model, dvalid)
print(acc_train)
print(acc_valid)


In [0]:
model.eval()
outputs = model(dvalid[0].unsqueeze(0), masked_lm_labels=dvalid[0].unsqueeze(0))
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
print(predicted_index)
print(dvalid[0])

In [0]:
# debug
model.train()
optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
for i in range(10):
    optimizer.zero_grad()
#     select_idx = 1
#     input_ids = tokens_tensor[select_idx].unsqueeze(0)
#     input_ids = tokens_tensor[0:3]
    input_ids = dtrain[0].unsqueeze(0)

    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    print(i, loss.item())
    loss.backward()
    optimizer.step()
    
#     loss.backward(retain_graph=True)



In [0]:
model.eval()
select_idx = 5
input_ids = tokens_tensor[select_idx].unsqueeze(0)
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(predicted_index)

In [0]:
# evaluate on random data to ensure accuracy metric works
model.eval()

np.random.seed(2019)
noise = np.random.randint(1, 10, 512).reshape(1,-1)
input_ids = torch.from_numpy(noise).to('cuda')

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(input_ids)
print(predicted_index)

In [0]:
# fix forward mask
model.eval()

np.random.seed(2019)
noise = np.random.randint(1, 10, 512).reshape(1,-1)
input_ids = torch.from_numpy(noise).to('cuda')
input_ids[0,0] = -inf

attention_mask = torch.from_numpy(np.ones(512).reshape(1,-1)).float().to('cuda')
attention_mask[0,0] = 0
# print(attention_mask)

token_type_ids = torch.from_numpy(np.zeros(512).reshape(1,-1)).long().to('cuda')

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]

predicted_index = torch.argmax(prediction_scores, dim=2)
a = evaluate(model, tokens_tensor[select_idx].unsqueeze(0))
print(a)
# print(predicted_index.shape)
print(input_ids)
print(predicted_index)

In [0]:
del model
torch.cuda.empty_cache()

In [0]:
# train model
import torch.optim as optim
import torch.nn.functional as F

def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()  # set to training mode, disappointingly does not actually train the model 
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            
optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
optimizer.zero_grad()
# output = model(tokens_tensor)
# loss = F.nll_loss(output, target.to('cuda'))
print([x for x in model.parameters()])


In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores)

In [0]:
print(input_ids.shape)
print(tokens_tensor[0].unsqueeze(0).shape)
print(tokens_tensor[0:2].shape)

In [0]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)
model.to('cuda')

# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
# input_ids = tokens_tensor[0].unsqueeze(0)
input_ids = tokens_tensor[0:2]

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores.shape)

for i in range(2):
    input_ids = tokens_tensor[0:2]
    outputs = model(input_ids, masked_lm_labels=input_ids)
    loss, prediction_scores = outputs[:2]
    loss.backward()
    print(loss)
    print(prediction_scores.shape)
#     loss.backward(retain_graph=True)

In [0]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.NONE)


# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = 8
# tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [0]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)

    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

predicted_index = torch.argmax(predictions[0, 11]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [0]:
for i in range(len(indexed_tokens)):
    predicted_index = torch.argmax(predictions[0, i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_token)

In [0]:
predicted_token = tokenizer.convert_ids_to_tokens([103])[0]
print(predicted_token)

In [0]:
import logging
logging.basicConfig(level=logging.INFO)

In [0]:
# example tokenization
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

masked_index = 8
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
print(tokens_tensor.shape)

In [0]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [0]:
# GPT-2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2-large')


# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
model.to('cuda')

100%|██████████| 3247202234/3247202234 [03:41<00:00, 14645396.05B/s]


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [0]:
# original_text = "Martin Steinegger is in Peru because he "
# original_text = "The real reason Steven keeps recruiting German postdocs is "
# original_text = "The secret to giving a fun and compelling Joint Lab Meeting presentation is "
# original_text = "UC Berkeley is"
# original_text = "Why did Donald Trump "
# original_text = "Finding genes is easy... The secret is "
# original_text = "Computational gene finding is easy, the problem is "
# original_text = "The future of Biomedical Engineering is "
# original_text = "The best way to describe how neural networks work is "
original_text = "Improving on state-of-the-art bacterial gene finding programs is hard, "


text = original_text
for i in range(20): # not the best way to iterate, but it works
    if text[-1] != ".":
        # Encode a text inputs
        indexed_tokens = tokenizer.encode(text)

        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])

        # If you have a GPU, put everything on cuda
        tokens_tensor = tokens_tensor.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word (in our case, the word 'man')
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

        text = predicted_text

In [0]:
print("Original text:\t\t", original_text)
print("Completed sentence:\t", predicted_text)

Original text:		 Improving on state-of-the-art bacterial gene finding programs is hard, 
Completed sentence:	 Improving on state-of-the-art bacterial gene finding programs is hard, but it's not impossible.
